In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns

In [66]:
# train = pd.read_csv("https://raw.githubusercontent.com/AMOGHA1140/kaggle_contents/main/titanic/train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/AMOGHA1140/kaggle_contents/main/titanic/test.csv")


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [6]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [72]:
train = pd.read_csv("https://raw.githubusercontent.com/AMOGHA1140/kaggle_contents/main/titanic/train.csv")

def preprocess(data):

  #drop ticket and name as they're useless
  data = data.drop(['Ticket', 'Name', 'Cabin'], axis=1)

  #remove the only 2 existing null values from Embarked
  data = data.dropna(subset=['Embarked'])

  #fill empty values of age and fare
  data['Age'] = data['Age'].fillna(data['Age'].median())
  data['Fare'].fillna(data['Fare'].median(), inplace=True)


  #one hot encode Sex and Embarked
  encoded_data = pd.get_dummies(data[['Sex', 'Embarked']])

  data = pd.concat([data, encoded_data], axis=1)
  data = data.drop(['Sex', 'Embarked'], axis=1)


  #normalize into gaussian distribution
  data['Fare'] = np.log(data['Fare'] + 1)
  data['SibSp'] = np.log(data['SibSp'] + 0.1)

  #scale the data
  scaler = StandardScaler()
  data['Age'] = scaler.fit_transform(data[['Age']])
  data['Fare'] = scaler.fit_transform(data[['Fare']])

  return data


train = preprocess(train)

# test = preprocess(test)
# print(train.columns)
train.head()


<ipython-input-72-84c23c5ffd92>:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Fare'].fillna(data['Fare'].median(), inplace=True)


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,-0.563674,0.095310,0,-0.877591,False,True,False,False,True
1,2,1,1,0.669217,0.095310,0,1.366380,True,False,True,False,False
2,3,1,3,-0.255451,-2.302585,0,-0.796281,True,False,False,False,True
3,4,1,1,0.438050,0.095310,0,1.066796,True,False,False,False,True
4,5,0,3,0.438050,-2.302585,0,-0.781901,False,True,False,False,True


In [73]:
X_train, X_test, y_train, y_test = train_test_split(train.drop(['Survived', 'PassengerId'], axis=1), train['Survived'], test_size=0.2, random_state=42)

In [74]:
model = LogisticRegression()
model.fit(X_train, y_train)

predictions = model.predict(X_test)
accuracy_score(y_test, predictions)



0.7808988764044944

In [76]:
test = pd.read_csv("https://raw.githubusercontent.com/AMOGHA1140/kaggle_contents/main/titanic/test.csv")

test = preprocess(test)
submissions = model.predict(test.drop('PassengerId', axis=1))


<ipython-input-72-84c23c5ffd92>:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Fare'].fillna(data['Fare'].median(), inplace=True)


In [77]:
df = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': submissions})
df.to_csv('submission2.csv', index=False)

In [78]:
def preprocess(data):

  #remove useless data
  try:
    data.drop(["Name", "Ticket", "Cabin"], axis=1, inplace=True)
  except:
    None

  #one-hot-encode Sex and Embarked
  # enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
  encoded_data = pd.get_dummies(data[['Sex', 'Embarked']])
  data = pd.concat([data, encoded_data], axis=1)
  data.drop(['Sex', 'Embarked'], axis=1, inplace=True)
  # data['Embarked'] = enc.fit_transform(data[['Embarked']])

  #scale
  data['Age'].fillna(data['Age'].median(), inplace=True)
  data['Fare'].fillna(data['Fare'].median(), inplace=True)

  scaler = StandardScaler()
  data['Age'] = scaler.fit_transform(data[['Age']])
  data['Fare'] = scaler.fit_transform(data[['Fare']])

  return data



train = preprocess(train)
# train.describe()
train.head()
# train.columns'

KeyError: "None of [Index(['Sex', 'Embarked'], dtype='object')] are in the [columns]"

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train.drop(['Survived', 'PassengerId'], axis=1), train['Survived'], test_size=0.2, random_state=42)

In [ ]:
model1 = LogisticRegression()
model1.fit(X_train, y_train)
predictions = model1.predict(X_test)

print(len(predictions))

179


In [ ]:
print(f"Accuracy: {accuracy_score(y_test, predictions)}")
print(f"F1 Score: {f1_score(y_test, predictions)}")

Accuracy: 0.8100558659217877
F1 Score: 0.7638888888888888


In [ ]:
test = preprocess(test)
test.head()

<ipython-input-39-ea28c8a80be6>:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Age'].fillna(data['Age'].mean(), inplace=True)
<ipython-input-39-ea28c8a80be6>:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', tr

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,892,3,0.334993,0,0,-0.498407,False,True,False,True,False
1,893,3,1.325530,1,0,-0.513274,True,False,False,False,True
2,894,2,2.514175,0,0,-0.465088,False,True,False,True,False
3,895,3,-0.259330,0,0,-0.483466,False,True,False,False,True
4,896,3,-0.655545,1,1,-0.418471,True,False,False,False,True


In [ ]:
submission = model1.predict(test.drop('PassengerId', axis=1))

submission = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': submission})
submission.to_csv('submission.csv', index=False)
print(submission)

     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         0
4            896         1
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]
